# Notebook 5 · Multi-Agent Routing

Many real deployments maintain multiple retrieval corpora. This notebook illustrates how to route a query between specialised retrievers before synthesising the final answer.

In [ ]:
from typing import Dict

from langchain.agents import Tool, initialize_agent
from langchain.schema import BaseRetriever
from langchain_openai import ChatOpenAI as LangChainChatOpenAI

from pprint import pprint

from shared import (
    DEFAULT_MODEL,
    RetrievalContext,
    build_baseline_chain,
    build_retrieval_context,
    pretty_print_json,
    time_execution,
)


In [ ]:
context = build_retrieval_context(top_k=4)
secondary_context = build_retrieval_context(top_k=2)

retriever_map: Dict[str, BaseRetriever] = {
    'support': context.retriever,
    'product': secondary_context.retriever,
}

def route_query(query: str) -> str:
    if 'price' in query.lower() or 'billing' in query.lower():
        return 'support'
    return 'product'


In [ ]:
def make_tool(name: str, retriever: BaseRetriever) -> Tool:
    def _search(q: str) -> str:
        docs = retriever.get_relevant_documents(q)
        return '\n\n'.join(doc.page_content for doc in docs) or 'No results.'

    return Tool(name=name, func=_search, description=f'Query the {name} knowledge base.')

router_tools = [make_tool(name, retriever) for name, retriever in retriever_map.items()]
policy_llm = LangChainChatOpenAI(model=DEFAULT_MODEL, temperature=0.1)
agent = initialize_agent(router_tools, policy_llm, agent='openai-functions')


In [ ]:
def answer(query: str) -> str:
    destination = route_query(query)
    print(f'Routing to: {destination}')
    response = agent.invoke({'input': query})
    return response['output']

print(answer('Outline the billing differences between Team and Enterprise tiers.'))


## Discussion

The simple keyword policy here is intentionally naive. Replace it with a classifier LLM call or a rules engine (e.g. spaCy, Hugging Face text classifier) to achieve more nuanced routing.